In [ ]:
!pip uninstall -y numpy numba tensorflow torch torchvision torchaudio torchdata torchtext dopamine-rl fastai librosa

# Instructions
This notebook allows you to load and evaluate a huggingface model on a subset of BLiMP (a linguistic acceptability judgment dataset) and GLUE (a natural language understanding benchmark collection). It is HIGHLY recommended to clone the GitHub repository and evaluate your model in the command-line; this will give you more freedom in the kinds of models you can evaluate. However, Colab provides a GPU that will allow you to load and evaluate smaller models.

To use this notebook:

1. Start by making a copy of this notebook so that you can make edits and run the code: File > Save a copy in Drive.

2. Set Runtime > Change runtime type > Hardware accelerator to GPU if it isn't already.

3. Run the setup script to install the required packages for evaluating.

4. Upload your model to the colab in the `/content/model_folder/` directory. This folder should include the following files, and probably a couple more depending on the type of model and tokenizer you use:
* `config.json`
* `pytorch_model.bin`
* `tokenizer_config.json`
* `vocab.json`

  a. To obtain these files given your pre-trained model and your tokenizer, load them using huggingface `transformers` and save them using these commands:
```
tokenizer.save_pretrained("./model_dir")
model.save_pretrained("./model_dir")
```
  b. Then, upload all the contents of `model_dir` (including any other files not mentioned above) to the `model_folder` folder in this Colab.

5. Choose the proper model type in the dropdown in the "load model and evaluate" cell. Use "decoder" for autoregressive (sometimes called "causal") language models, like GPT/OPT; "encoder" for masked language models, like BERT/RoBERTa; or "encoder-decoder" for text-to-text models, like T5/BART.

6. Run the cells below to load and evaluate your model.

In [ ]:
#@title Setup script { display-mode: "form" }
#@markdown Run this cell to install the necessary packages (may take a few minutes).
%%shell
# Remove previous installation if it exists
cd /content
mkdir model_folder
pip uninstall -y lm-eval
rm -rf evaluation-pipeline/

# Install evaluation-pipeline
git clone -b colab https://github.com/babylm/evaluation-pipeline &> /dev/null
cd evaluation-pipeline/
pip install -e ".[colab]"
# Install other necessary packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 'numpy>=1.24.0' lm_eval --extra-index-url https://download.pytorch.org/whl/cu113

# Unpack dataset
unzip filter_data.zip

**Restart runtime and continue from here!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation

In [ ]:
#@title Load model and evaluate (BLiMP) { display-mode: "form" }
model = "model_dir" #@param {"type": "string"}
model_type = "encoder" #@param ["decoder", "encoder", "encoder-decoder"]
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
%run /content/evaluation-pipeline/babylm_eval.py \
  "$model" \
  "$model_type" \
  -t "blimp"

In [ ]:
#@title Load model and evaluate ((Super)GLUE) { display-mode: "form" }
#@markdown Run this cell to fine-tune your model on (Super)GLUE tasks.
#@markdown We provide some default hyperparameters that you may adjust.
model = "model_dir" #@param {"type": "string"}
learning_rate = 5e-5 #@param {"type": "number"}
batch_size = 64 #@param {"type": "integer"}
eval_every = 200 #@param {"type": "integer"}
patience = 10 #@param {"type": "integer"}
max_epochs = 10 #@param {"type": "integer"}
seed = 12 #@param {"type": "integer"}
# file_name = "examples3.csv" #@param {"type": "string"}
# model_names = ["opt-125m", "opt-350m", "opt-1.3b", "opt-2.7b"] #@param {"type": "raw"}

%cd /content/evaluation-pipeline
!./finetune_all_tasks.sh \
    "$model" \
    "$learning_rate" \
    "$patience" \
    "$batch_size" \
    "$eval_every" \
    "$max_epochs" \
    "$seed"